# Neuroglancer Builder — General (OME‑TIFF, TIFF, or BIGTIFF 2D histology data → 4 split layers)

This notebook builds four single‑channel **Neuroglancer precomputed** layers (**DAPI**, **aGFP**, **aRFP**, **aTH**) from your QuPath‑exported OME‑TIFFs.

**Highlights**
- Finds and **sorts sections** so that `s1-s2` comes **before** `s2-s3`, then by plate (P1..), row (A/B/C), column (1..4).
- Creates **one layer per channel**, on your **D:\\** drive (Windows path safe `file://D:/...`).
- Writes all tiles, **padding with zeros** outside each section’s bounds (no more 404s).
- Optional **pre-seed** step writes zeros for *every* tile first, so even if a tile’s data is entirely outside bounds, a file still exists.
- Includes a quick **post-check** for missing tiles, and a ready-to-use local server + Neuroglancer URL snippet.

> Run top-to-bottom after editing the **Config** cell.


# Uncompress exported QuPath TIF

*** Adjust these SRC/DST paths only, then copy and paste entire script into command prompt ***

```bat
set "SRC=F:\QuPath_cjSwan-Endogenous\exports_fullres_ome_clean"
set "DST=F:\QuPath_cjSwan-Endogenous\exports_fullres_ome_uncompressed"

mkdir "%DST%" 2>nul

echo === Rewriting OME-TIFFs as series 0, Uncompressed, BigTIFF, tiled 512x512 ===
for %F in ("%SRC%\*.ome.tif" "%SRC%\*.ome.tiff") do (
  echo.
  echo -> %~nxF
  "C:\bftools\bfconvert.bat" -no-upgrade -overwrite ^
    -series 0 ^
    -bigtiff ^
    -tilex 512 -tiley 512 ^
    -compression Uncompressed ^
    "%~fF" "%DST%\%~nxF"
)

echo.
echo === Verifying with showinf (nonzero exit => BAD) ===
for %F in ("%DST%\*.ome.tif" "%DST%\*.ome.tiff") do @("C:\bftools\showinf.bat" -no-upgrade -nopix "%~fF" >nul 2>&1) || echo BAD: %~nxF

echo.
echo Done. Clean files in:
echo   %DST%
```


In [1]:
# Imports & versions
import sys, os, re, json, math, numpy as np, tifffile as tiff, zarr
from pathlib import Path
from cloudvolume import CloudVolume

print('Python:', sys.version)
print('NumPy:', np.__version__)
print('tifffile:', tiff.__version__)
import imagecodecs, numcodecs
print('imagecodecs:', getattr(imagecodecs, '__version__', 'n/a'))
print('zarr:', zarr.__version__)
print('numcodecs:', numcodecs.__version__)
try:
    import tqdm; print('tqdm:', tqdm.__version__)
except Exception:
    print('tqdm: not installed (optional)')


Python: 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
NumPy: 2.0.2
tifffile: 2024.8.30
imagecodecs: 2021.8.26
zarr: 2.18.2
numcodecs: 0.12.1
tqdm: 4.62.3


In [2]:
# =====================
# Config (EDIT THESE) |
# =====================
INPUT_DIR  = r'D:\BCH-cjDAE8-mHGHpA\cjSwan\Confocal\Endogenous'
OUTPUT_DIR = r'D:\BCH-cjDAE8-mHGHpA\cjSwan\cjSwan-confocal-endogenous_neuroglancer_dataset_per_section'

# Channel & tiling
LAYER_NAMES = ['DAPI', 'EGFP', 'dTom', 'aTH'] 
# LAYER_NAMES = ['DAPI', 'aTH', 'dTom', 'aRFP']     # Channels 0..3 in your OME-TIFF
# LAYER_NAMES = ["DAPI","aGFP","aRFP","aTH"]
TILE_YX     = 2048                                # XY tile edge length
CHUNK_Z     = 1
ENCODING    = 'raw'                               # or 'gzip' (slower, smaller)
DTYPE       = 'uint16'

# Resolution in nanometers (XY pixel size; Z spacing between sections)
RES_XY_NM   = 1000
RES_Z_NM    = 100000

# Pre-seed zeros for every tile? (Recommended: True)
PRESEED_ALL_TILES = True

# =====================
# No edits below here  |
# =====================
OUTPUT_ROOT = Path(OUTPUT_DIR) / 'layers_split'
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

print('INPUT_DIR :', INPUT_DIR)
print('OUTPUT_DIR:', OUTPUT_DIR)
print('Layers    :', LAYER_NAMES)


INPUT_DIR : D:\BCH-cjDAE8-mHGHpA\cjSwan\Confocal\Endogenous
OUTPUT_DIR: D:\BCH-cjDAE8-mHGHpA\cjSwan\cjSwan-confocal-endogenous_neuroglancer_dataset_per_section
Layers    : ['DAPI', 'EGFP', 'dTom', 'aTH']


In [3]:
import re
import string
import tifffile as tiff
from pathlib import Path
import xml.etree.ElementTree as ET
from typing import Optional, Tuple

def discover_files(d):
    """Legacy helper: return only OME-TIFFs"""
    p = Path(d)
    files = [str(x) for x in p.glob('*.ome.tif')]
    files += [str(x) for x in p.glob('*.ome.tiff')]
    return files

# Extended regex: supports plate-row-col format AND fallback for channel-index BigTIFFs
_name_pat = re.compile(
    r'(?:P(?P<plate>\d+)-(?P<row>[A-Z])(?P<col>\d)_s(?P<slo>\d)-s(?P<shi>\d))|'
    r'(?:_(?P<ch>\d+)_CH\.btf)$',
    re.IGNORECASE
)

def section_sort_key(path_or_name: str):
    """Sorting key based on plate/row/col or channel index; fallback to filename."""
    name = Path(path_or_name).name
    m = _name_pat.search(name)
    if not m:
        return (999, 'Z', 99, 99, name.lower())

    gd = m.groupdict()
    if gd.get('plate'):
        plate = int(gd['plate'])
        row   = gd['row'].upper()
        col   = int(gd['col'])
        slo   = int(gd['slo'])
        shi   = int(gd['shi'])
        return (slo, shi, plate, row, col, name.lower())
    elif gd.get('ch'):
        ch = int(gd['ch'])
        return (0, 0, 0, 'A', ch, name.lower())
    else:
        return (999, 'Z', 99, 99, name.lower())

def file_cloudpath(p: Path) -> str:
    p = Path(p).resolve()
    drive = p.drive.rstrip(':')
    tail  = p.as_posix().split(':', 1)[1]
    return f'file://{drive}:{tail}'

def ensure_uint16(arr):
    if arr.dtype == np.uint16:
        return arr
    return arr.astype(np.uint16, copy=False)

def list_all_tifs(input_dir: str):
    """Return unique list of OME, plain, and BigTIFFs in input_dir, sorted by section_sort_key."""
    p = Path(input_dir)
    seen = {}
    for pat in ("*.ome.tif", "*.ome.tiff", "*.tif", "*.tiff", "*.btf", "*.btiff"):
        for x in p.glob(pat):
            seen[str(x)] = True
    files = list(seen.keys())
    return sorted(files, key=section_sort_key)

def xy_from_ome(tf: tiff.TiffFile) -> Tuple[Optional[int], Optional[int]]:
    """Try to read SizeX/SizeY from OME-XML (returns (X,Y) or (None,None))."""
    try:
        xml = getattr(tf, 'ome_metadata', None)
        if not xml:
            return None, None
        root = ET.fromstring(xml)
        if '}' in root.tag:
            ns = {'ome': root.tag.split('}')[0].strip('{')}
            pixels = root.find('.//ome:Pixels', ns)
        else:
            pixels = root.find('.//Pixels')
        if pixels is None:
            return None, None
        sx = pixels.get('SizeX')
        sy = pixels.get('SizeY')
        return (int(sx) if sx else None, int(sy) if sy else None)
    except Exception:
        return None, None


In [4]:
# --- discover files (OME + plain + BigTIFF) ---
files = list_all_tifs(INPUT_DIR)
print(f'Discovered {len(files)} TIFF(s) (OME, plain, BigTIFF)')

if not files:
    raise SystemExit('No TIFFs found. Check INPUT_DIR.')

maxY = maxX = 0
bad = []
fallback_used = []

for f in files:
    name = Path(f).name
    # optionally skip label images
    if ' label' in name.lower():
        continue
    try:
        with tiff.TiffFile(f) as tf:
            try:
                # Preferred: series[0] (works for many OME, plain, and BigTIFFs)
                ser  = tf.series[0]
                axes = ser.axes.upper()
                if 'Y' in axes and 'X' in axes:
                    Y = ser.shape[axes.find('Y')]
                    X = ser.shape[axes.find('X')]
                else:
                    raise RuntimeError(f"series[0] axes lack X/Y: {axes}")
            except Exception:
                # Fallback 1: OME-XML
                X, Y = xy_from_ome(tf)
                if X is not None and Y is not None:
                    fallback_used.append(name)
                else:
                    # Fallback 2: first page dimensions (works for plain & BigTIFFs)
                    try:
                        p0 = tf.pages[0]
                        X, Y = int(p0.imagewidth), int(p0.imagelength)
                        fallback_used.append(name)
                    except Exception as e2:
                        bad.append((name, f"no XY via series/OME/page0 ({e2})"))
                        continue  # skip this file for sizing

            maxY = max(maxY, int(Y))
            maxX = max(maxX, int(X))
    except Exception as e:
        bad.append((name, f"open error: {e}"))

if fallback_used:
    print(f"WARN: Used fallback sizing for {len(fallback_used)} file(s), e.g.:")
    for n in fallback_used[:5]:
        print(" -", n)

if bad:
    print('WARN: Some files were skipped for sizing:')
    for n, msg in bad[:10]:
        print(' -', n, '->', msg)

Z = len(files)
print(f'Target volume: X={maxX}, Y={maxY}, Z={Z}')


Discovered 9 TIFF(s) (OME, plain, BigTIFF)
Target volume: X=4063, Y=3050, Z=9


In [5]:
def make_layer(path: Path, size_xyz, chunk_xy, enc=ENCODING, dtype=DTYPE):
    info = CloudVolume.create_new_info(
        num_channels=1,
        layer_type='image',
        data_type=dtype,
        encoding=enc,
        resolution=[RES_XY_NM, RES_XY_NM, RES_Z_NM],
        voxel_offset=[0, 0, 0],
        chunk_size=[chunk_xy, chunk_xy, CHUNK_Z],
        volume_size=list(size_xyz),
    )
    cp = file_cloudpath(path)
    print('Creating layer at:', cp)
    vol = CloudVolume(cp, info=info, progress=False)
    vol.commit_info()
    vol.commit_provenance()
    infopath = path / 'info'
    print('  info exists:', infopath.exists(), '->', infopath)
    return vol

X, Y = maxX, maxY
layer_paths = [OUTPUT_ROOT / name for name in LAYER_NAMES]
vols = [make_layer(p, (X, Y, len(files)), TILE_YX) for p in layer_paths]
vol_by_name = dict(zip(LAYER_NAMES, vols))
print('Layers ready:', [str(p) for p in layer_paths])


Creating layer at: file://D:/BCH-cjDAE8-mHGHpA/cjSwan/cjSwan-confocal-endogenous_neuroglancer_dataset_per_section/layers_split/DAPI
  info exists: True -> D:\BCH-cjDAE8-mHGHpA\cjSwan\cjSwan-confocal-endogenous_neuroglancer_dataset_per_section\layers_split\DAPI\info
Creating layer at: file://D:/BCH-cjDAE8-mHGHpA/cjSwan/cjSwan-confocal-endogenous_neuroglancer_dataset_per_section/layers_split/EGFP
  info exists: True -> D:\BCH-cjDAE8-mHGHpA\cjSwan\cjSwan-confocal-endogenous_neuroglancer_dataset_per_section\layers_split\EGFP\info
Creating layer at: file://D:/BCH-cjDAE8-mHGHpA/cjSwan/cjSwan-confocal-endogenous_neuroglancer_dataset_per_section/layers_split/dTom
  info exists: True -> D:\BCH-cjDAE8-mHGHpA\cjSwan\cjSwan-confocal-endogenous_neuroglancer_dataset_per_section\layers_split\dTom\info
Creating layer at: file://D:/BCH-cjDAE8-mHGHpA/cjSwan/cjSwan-confocal-endogenous_neuroglancer_dataset_per_section/layers_split/aTH
  info exists: True -> D:\BCH-cjDAE8-mHGHpA\cjSwan\cjSwan-confocal-endo

In [6]:
# Pre-seed zeros across all tiles for all 4 layers with correct XY ordering
import math, numpy as np
from pathlib import Path
from cloudvolume import CloudVolume

ROOT = Path(OUTPUT_DIR) / "layers_split"
LAYER_NAMES = ['DAPI', 'EGFP', 'dTom', 'aTH'] 
# LAYER_NAMES = ['DAPI', 'aTH', 'dTom', 'aRFP']
# LAYER_NAMES = ["DAPI","aGFP","aRFP","aTH"]

# Open layers (Windows-safe file://D:/... path)
vols = [CloudVolume("file://" + (ROOT/name).as_posix(), progress=False) for name in LAYER_NAMES]

Z = len(files)
tilesY = math.ceil(maxY / TILE_YX)
tilesX = math.ceil(maxX / TILE_YX)

print(f"Tiling grid: {tilesY} × {tilesX} of {TILE_YX}")
print("Pre-seeding zeros across all tiles (all layers)… this can take a while.")

for zi in range(Z):
    for ty in range(tilesY):
        y0, y1 = ty*TILE_YX, min(maxY, (ty+1)*TILE_YX)
        for tx in range(tilesX):
            x0, x1 = tx*TILE_YX, min(maxX, (tx+1)*TILE_YX)
            bx, by = x1 - x0, y1 - y0

            # IMPORTANT: pad is (X, Y, Z=1) — NOT (Y, X, 1)
            pad = np.zeros((bx, by, 1), dtype=np.uint16)

            # Write to each single-channel layer
            for vol in vols:
                vol[x0:x1, y0:y1, zi:zi+1] = pad

        print(f"  z={zi+1}/{Z} row {ty+1}/{tilesY}")
print("Pre-seed done.")


Tiling grid: 2 × 2 of 2048
Pre-seeding zeros across all tiles (all layers)… this can take a while.
  z=1/9 row 1/2
  z=1/9 row 2/2
  z=2/9 row 1/2
  z=2/9 row 2/2
  z=3/9 row 1/2
  z=3/9 row 2/2
  z=4/9 row 1/2
  z=4/9 row 2/2
  z=5/9 row 1/2
  z=5/9 row 2/2
  z=6/9 row 1/2
  z=6/9 row 2/2
  z=7/9 row 1/2
  z=7/9 row 2/2
  z=8/9 row 1/2
  z=8/9 row 2/2
  z=9/9 row 1/2
  z=9/9 row 2/2
Pre-seed done.


In [7]:
print('Writing tiles to layers (series[0], channels 0..3)…')

for zi, f in enumerate(files):
    print(f'[Z {zi+1}/{len(files)}] {Path(f).name}')
    with tiff.TiffFile(f) as tf:
        ser  = tf.series[0]
        axes = ser.axes.upper()
        zr   = zarr.open(ser.aszarr(), mode='r')
        iY, iX, iC = axes.find('Y'), axes.find('X'), axes.find('C')
        if iY == -1 or iX == -1:
            raise RuntimeError(f'Missing X/Y axes in series[0]: {f}')
        Yc, Xc = ser.shape[iY], ser.shape[iX]
        hasC = (iC != -1)

        for ty in range(tilesY):
            y0, y1 = ty*TILE_YX, min(Y, (ty+1)*TILE_YX)
            for tx in range(tilesX):
                x0, x1 = tx*TILE_YX, min(X, (tx+1)*TILE_YX)
                tile = np.zeros((y1-y0, x1-x0, 4), dtype=np.uint16)
                if y0 < Yc and x0 < Xc:
                    ys, xs = min(y1, Yc), min(x1, Xc)
                    sl = [slice(None)] * zr.ndim
                    sl[iY] = slice(y0, ys)
                    sl[iX] = slice(x0, xs)
                    if hasC:
                        sl[iC] = slice(0,4)
                        src = np.asarray(zr[tuple(sl)])
                        src_yxc = np.transpose(src, (iY, iX, iC))
                        h, w, c = src_yxc.shape
                        tile[:h, :w, :min(4,c)] = ensure_uint16(src_yxc[..., :4])
                    else:
                        src = np.asarray(zr[tuple(sl)])
                        h, w = src.shape
                        tile[:h, :w, 0] = ensure_uint16(src)
                # Write channels
                for ci, lname in enumerate(LAYER_NAMES):
                    vol = vol_by_name[lname]
                    arr_xy = tile[..., ci].T
                    vol[x0:x1, y0:y1, zi:zi+1] = arr_xy[:, :, None]
            print(f'  wrote row {ty+1}/{tilesY}')
print('All tiles written.')


Writing tiles to layers (series[0], channels 0..3)…
[Z 1/9] Stitch_cjSwan-LH_P1-A2_S2_DA8-EGFP_DA8-dTom_aTH-647_Cycle.tif
  wrote row 1/2
  wrote row 2/2
[Z 2/9] Stitch_cjSwan-LH_P1-B2_S2_DA8-EGFP_DA8-dTom_aTH-647_Cycle.tif
  wrote row 1/2
  wrote row 2/2
[Z 3/9] Stitch_cjSwan-LH_P1-C2_S2_DA8-EGFP_DA8-dTom_aTH-647_Cycle.tif
  wrote row 1/2
  wrote row 2/2
[Z 4/9] Stitch_cjSwan-LH_P2-A2_S2_DA8-EGFP_DA8-dTom_aTH-647_Cycle.tif
  wrote row 1/2
  wrote row 2/2
[Z 5/9] Stitch_cjSwan-LH_P2-B2_S2_DA8-EGFP_DA8-dTom_aTH-647_Cycle.tif
  wrote row 1/2
  wrote row 2/2
[Z 6/9] Stitch_cjSwan-LH_P2-C2_S2_DA8-EGFP_DA8-dTom_aTH-647_Cycle.tif
  wrote row 1/2
  wrote row 2/2
[Z 7/9] Stitch_cjSwan-LH_P3-A1_S2_DA8-EGFP_DA8-dTom_aTH-647_Cycle.tif
  wrote row 1/2
  wrote row 2/2
[Z 8/9] Stitch_cjSwan-LH_P3-B1_S2_DA8-EGFP_DA8-dTom_aTH-647_Cycle.tif
  wrote row 1/2
  wrote row 2/2
[Z 9/9] Stitch_cjSwan-LH_P3-C1_S2_DA8-EGFP_DA8-dTom_aTH-647_Cycle.tif
  wrote row 1/2
  wrote row 2/2
All tiles written.


In [12]:
# Build MIP1..MIP4 for all layers (RAW tiles, no gzip).
# Neuroglancer will automatically switch to higher MIP indices when you zoom out.

import json, math, numpy as np
from pathlib import Path
from cloudvolume import CloudVolume

# --- config ---
MAX_MIPS   = 4            # build MIP1..MIP4 (increase to 5/6 if still heavy)
CHUNK_XY   = 2048         # keep same chunk size across MIPs
LAYER_NAMES = ['DAPI', 'EGFP', 'dTom', 'aTH'] 
# LAYER_NAMES = ["DAPI","EGFP","dTom","aTH"]
# LAYER_NAMES = ['DAPI', 'aTH', 'dTom', 'aRFP']

ROOT = Path(OUTPUT_DIR) / "layers_split"

def file_cloudpath(p: Path) -> str:
    return "file://" + str(Path(p).resolve()).replace("\\", "/")

def ceil_div(a,b): return (a + b - 1) // b

def blk_to_xy(blk: np.ndarray) -> np.ndarray:
    """Normalize CloudVolume block (X,Y,1) or (X,Y,1,1) to 2D (X,Y)."""
    arr = np.asarray(blk)
    if arr.ndim == 2:
        return arr
    # collapse trailing axes and take the first slice
    arr = arr.reshape(arr.shape[0], arr.shape[1], -1)
    return arr[..., 0]

def block_mean_2x(src_xy: np.ndarray) -> np.ndarray:
    """Downsample 2D (X,Y) by 2 via 2×2 mean, pad edges by extension if odd."""
    if src_xy.ndim != 2:
        raise ValueError(f"block_mean_2x expected 2D input, got {src_xy.shape}")
    X, Y = src_xy.shape
    padX = X & 1; padY = Y & 1
    if padX or padY:
        src_xy = np.pad(src_xy, ((0,padX),(0,padY)), mode='edge')
        X, Y = src_xy.shape
    ds = (
        src_xy[0:X:2, 0:Y:2].astype(np.uint32)
      + src_xy[1:X:2, 0:Y:2].astype(np.uint32)
      + src_xy[0:X:2, 1:Y:2].astype(np.uint32)
      + src_xy[1:X:2, 1:Y:2].astype(np.uint32)
    ) // 4
    return ds.astype(np.uint16)

def load_info(layer_path: Path) -> dict:
    """Read info; if missing/empty, bootstrap from CloudVolume mip 0."""
    info_path = layer_path / "info"
    if not info_path.exists() or info_path.stat().st_size == 0:
        # bootstrap from the existing dataset on disk
        vol0 = CloudVolume(file_cloudpath(layer_path), mip=0, progress=False, compress=False)
        info = vol0.info.copy()
        # Ensure MIP0 encoding is raw
        info["scales"][0]["encoding"] = "raw"
        info["scales"][0]["chunk_sizes"] = [[int(CHUNK_XY), int(CHUNK_XY), 1]]
        info_path.write_text(json.dumps(info, indent=2))
        print(f"[{layer_path.name}] bootstrapped info from dataset.")
        return info
    return json.loads(info_path.read_text())

def save_info(layer_path: Path, info: dict) -> None:
    (layer_path / "info").write_text(json.dumps(info, indent=2))

def ensure_raw_scales(info: dict, up_to_mip: int) -> dict:
    """Force encoding='raw' and ensure MIP entries up to up_to_mip exist with proper sizes/keys."""
    scales = info.get("scales", [])
    if not scales:
        raise RuntimeError("No 'scales' in info.")

    # Force raw + chunk size on existing scales
    for sc in scales:
        sc["encoding"] = "raw"
        sc["chunk_sizes"] = [[int(CHUNK_XY), int(CHUNK_XY), 1]]

    # Append scales until we have MIP up_to_mip
    while len(scales) <= up_to_mip:
        prev = scales[-1]
        res  = [int(prev["resolution"][0])*2, int(prev["resolution"][1])*2, int(prev["resolution"][2])]
        size = [ceil_div(int(prev["size"][0]),2), ceil_div(int(prev["size"][1]),2), int(prev["size"][2])]
        scales.append({
            "encoding": "raw",
            "chunk_sizes": [[int(CHUNK_XY), int(CHUNK_XY), 1]],
            "key": f"{res[0]}_{res[1]}_{res[2]}",
            "resolution": res,
            "size": size,
            "voxel_offset": prev.get("voxel_offset", [0,0,0]),
        })

    info["scales"] = scales[:up_to_mip+1]
    return info

def build_mips_for_layer(layer_path: Path):
    # 1) make sure info exists & is raw, extend scales to MAX_MIPS
    info = ensure_raw_scales(load_info(layer_path), MAX_MIPS)
    save_info(layer_path, info)

    s0 = info["scales"][0]
    X0, Y0, Z = map(int, s0["size"])
    print(f"Layer: {layer_path.name}  MIP0 size={[X0,Y0,Z]}  encoding={s0['encoding']}")

    # 2) build each MIP from the previous MIP
    for mip in range(1, MAX_MIPS+1):
        Xm, Ym, Zm = map(int, info["scales"][mip]["size"])
        print(f"  -> MIP{mip} target size={[Xm, Ym, Zm]}")

        vol_prev = CloudVolume(file_cloudpath(layer_path), mip=mip-1, progress=False, compress=False)
        vol_m    = CloudVolume(file_cloudpath(layer_path), mip=mip,   progress=False, compress=False)

        tilesX = math.ceil(Xm / CHUNK_XY)
        tilesY = math.ceil(Ym / CHUNK_XY)
        prev_size = list(map(int, vol_prev.info["scales"][mip-1]["size"]))

        for z in range(Zm):
            for ix in range(tilesX):
                x0m = ix * CHUNK_XY
                x1m = min(Xm, (ix+1)*CHUNK_XY)
                for iy in range(tilesY):
                    y0m = iy * CHUNK_XY
                    y1m = min(Ym, (iy+1)*CHUNK_XY)

                    # read 2x region from previous MIP (clamped)
                    X0a, X1a = x0m*2, min(prev_size[0], x1m*2)
                    Y0a, Y1a = y0m*2, min(prev_size[1], y1m*2)

                    blk = vol_prev[X0a:X1a, Y0a:Y1a, z:z+1]   # (X,Y,1) or (X,Y,1,1)
                    if blk.size == 0:
                        ds = np.zeros((x1m-x0m, y1m-y0m), np.uint16)
                    else:
                        src_xy = blk_to_xy(blk)
                        ds = block_mean_2x(src_xy)            # (Xa/2, Ya/2)
                        # pad/crop to exact tile dims
                        needX = (x1m-x0m) - ds.shape[0]
                        needY = (y1m-y0m) - ds.shape[1]
                        if needX > 0 or needY > 0:
                            ds = np.pad(ds, ((0,max(0,needX)), (0,max(0,needY))), mode='edge')
                        ds = ds[:(x1m-x0m), : (y1m-y0m)]

                    vol_m[x0m:x1m, y0m:y1m, z:z+1] = ds[:, :, None]
            print(f"    MIP{mip} z-slice {z+1}/{Zm} done")
        print(f"  MIP{mip} complete.")

# --- run all layers ---
for lname in LAYER_NAMES:
    build_mips_for_layer(ROOT / lname)

print("\n✅ Multiscale complete (MIP1..MIP4, RAW). Hard refresh Neuroglancer and try zooming out.")


Layer: DAPI  MIP0 size=[4063, 3050, 9]  encoding=raw
  -> MIP1 target size=[2032, 1525, 9]
    MIP1 z-slice 1/9 done
    MIP1 z-slice 2/9 done
    MIP1 z-slice 3/9 done
    MIP1 z-slice 4/9 done
    MIP1 z-slice 5/9 done
    MIP1 z-slice 6/9 done
    MIP1 z-slice 7/9 done
    MIP1 z-slice 8/9 done
    MIP1 z-slice 9/9 done
  MIP1 complete.
  -> MIP2 target size=[1016, 763, 9]
    MIP2 z-slice 1/9 done
    MIP2 z-slice 2/9 done
    MIP2 z-slice 3/9 done
    MIP2 z-slice 4/9 done
    MIP2 z-slice 5/9 done
    MIP2 z-slice 6/9 done
    MIP2 z-slice 7/9 done
    MIP2 z-slice 8/9 done
    MIP2 z-slice 9/9 done
  MIP2 complete.
  -> MIP3 target size=[508, 382, 9]
    MIP3 z-slice 1/9 done
    MIP3 z-slice 2/9 done
    MIP3 z-slice 3/9 done
    MIP3 z-slice 4/9 done
    MIP3 z-slice 5/9 done
    MIP3 z-slice 6/9 done
    MIP3 z-slice 7/9 done
    MIP3 z-slice 8/9 done
    MIP3 z-slice 9/9 done
  MIP3 complete.
  -> MIP4 target size=[254, 191, 9]
    MIP4 z-slice 1/9 done
    MIP4 z-slice 2/9 

In [13]:
def expected_chunk_names(size_x, size_y, size_z, cx, cy, cz):
    nx = math.ceil(size_x / cx); ny = math.ceil(size_y / cy)
    out = []
    for zi in range(size_z):
        for iy in range(ny):
            for ix in range(nx):
                x0 = ix*cx; x1 = min(size_x, (ix+1)*cx)
                y0 = iy*cy; y1 = min(size_y, (iy+1)*cy)
                out.append(f'{x0}-{x1}_{y0}-{y1}_{zi}-{zi+1}')
    return out, nx, ny

# info = json.loads((OUTPUT_ROOT / 'Nissl' / 'info').read_text())
info = json.loads((OUTPUT_ROOT / 'DAPI' / 'info').read_text())
scale = info['scales'][0]
scale_key = scale['key']
cx, cy, cz = map(int, scale['chunk_sizes'][0])
size_x, size_y, size_z = map(int, scale['size'])
expected, nx, ny = expected_chunk_names(size_x, size_y, size_z, cx, cy, cz)

print('scale_key:', scale_key, '| chunk:', (cx,cy,cz), '| size:', (size_x,size_y,size_z))
print('Expecting:', len(expected), 'files per layer')

missing_any = False
for lname in LAYER_NAMES:
    sdir = OUTPUT_ROOT / lname / scale_key
    present = set(p.name for p in sdir.glob('*_*_*') if p.is_file())
    missing = [n for n in expected if n not in present]
    if missing:
        missing_any = True
        print(f'[{lname}] MISSING {len(missing)} files (showing up to 5):', missing[:5])
    else:
        print(f'[{lname}] OK (all chunks present)')

if not missing_any:
    print('\nAll layers complete. You can serve & view in Neuroglancer.')
else:
    print('\nSome tiles are missing — re-run tiles cell or inspect specific coordinates.')


scale_key: 1000_1000_100000 | chunk: (2048, 2048, 1) | size: (4063, 3050, 9)
Expecting: 36 files per layer
[DAPI] OK (all chunks present)
[EGFP] MISSING 36 files (showing up to 5): ['0-2048_0-2048_0-1', '2048-4063_0-2048_0-1', '0-2048_2048-3050_0-1', '2048-4063_2048-3050_0-1', '0-2048_0-2048_1-2']
[dTom] OK (all chunks present)
[aTH] OK (all chunks present)

Some tiles are missing — re-run tiles cell or inspect specific coordinates.


In [14]:
# Decompress any .gz tiles (all MIPs, all layers) and force encoding='raw' in info.
# Safe to re-run; skips already-raw tiles and only updates what's needed.

import json, gzip, shutil
from pathlib import Path

# Uses your existing OUTPUT_DIR and LAYER_NAMES; tweak if needed
ROOT = Path(OUTPUT_DIR) / "layers_split"
try:
    LAYER_NAMES
except NameError:

    LAYER_NAMES = ['DAPI', 'EGFP', 'dTom', 'aTH'] 
#     LAYER_NAMES = ['DAPI', 'aTH', 'dTom', 'aRFP']
#     LAYER_NAMES = ["DAPI","aGFP","aRFP","aTH"]

def gunzip_to(src_gz: Path, dst_raw: Path):
    dst_raw.parent.mkdir(parents=True, exist_ok=True)
    with gzip.open(src_gz, 'rb') as fi, open(dst_raw, 'wb') as fo:
        shutil.copyfileobj(fi, fo)

for lname in LAYER_NAMES:
    layer = ROOT / lname
    info_path = layer / "info"
    if not info_path.exists():
        print(f"[{lname}] No info file at {info_path} — skipping.")
        continue

    info = json.loads(info_path.read_text())
    # Force encoding='raw' for every scale
    enc_changed = False
    for s in info.get("scales", []):
        if s.get("encoding", "raw") != "raw":
            s["encoding"] = "raw"
            enc_changed = True

    total_decompressed = 0
    # Walk each scale dir and gunzip tiles
    for s in info.get("scales", []):
        key = s["key"]
        sdir = layer / key
        if not sdir.exists():
            continue
        gz_files = list(sdir.rglob("*.gz"))
        if gz_files:
            print(f"[{lname}] {key}: decompressing {len(gz_files):,} tile(s)…")
        for i, gz in enumerate(gz_files, 1):
            raw = gz.with_suffix("")  # strip .gz
            if raw.exists():
                # A raw tile is already present; remove the gz to avoid ambiguity
                try:
                    gz.unlink()
                except Exception:
                    pass
                continue
            try:
                gunzip_to(gz, raw)
                gz.unlink()
                total_decompressed += 1
            except Exception as e:
                print(f"  WARN: failed to decompress {gz.name}: {e}")

            if i % 1000 == 0:
                print(f"  …{i} done")

    if enc_changed:
        info_path.write_text(json.dumps(info, indent=2))
        print(f"[{lname}] info updated -> encoding='raw' for all scales.")

    leftovers = list((layer).rglob("*.gz"))
    print(f"[{lname}] decompressed {total_decompressed} tile(s). Leftover .gz: {len(leftovers)}")

print("\nDone. If you’re using `http-server`, restart it or run with cache disabled (e.g. `http-server -c-1 --cors`). Hard refresh Neuroglancer.")


[DAPI] decompressed 0 tile(s). Leftover .gz: 0
[EGFP] 1000_1000_100000: decompressing 36 tile(s)…
[EGFP] decompressed 36 tile(s). Leftover .gz: 0
[dTom] decompressed 0 tile(s). Leftover .gz: 0
[aTH] decompressed 0 tile(s). Leftover .gz: 0

Done. If you’re using `http-server`, restart it or run with cache disabled (e.g. `http-server -c-1 --cors`). Hard refresh Neuroglancer.


In [15]:
import json
from pathlib import Path

# Point this to your dataset root (where the per-layer folders live)
ROOT = Path(OUTPUT_DIR) / "layers_split"
# LAYER_NAMES = ['DAPI', 'aTH', 'dTom', 'aRFP']
LAYER_NAMES = ['DAPI', 'EGFP', 'dTom', 'aTH'] 

for lname in LAYER_NAMES:
    info_path = ROOT / lname / "info"
    if not info_path.exists():
        print(f"SKIP: {info_path} not found")
        continue

    # Load JSON
    with open(info_path, "r") as f:
        info = json.load(f)

    # Add/overwrite the background color field
    info["default_background_color"] = [0, 0, 0, 0]

    # Save back to disk (pretty formatting so it's easy to inspect)
    with open(info_path, "w") as f:
        json.dump(info, f, indent=2)

    print(f"Patched: {info_path}")


Patched: D:\BCH-cjDAE8-mHGHpA\cjSwan\cjSwan-confocal-endogenous_neuroglancer_dataset_per_section\layers_split\DAPI\info
Patched: D:\BCH-cjDAE8-mHGHpA\cjSwan\cjSwan-confocal-endogenous_neuroglancer_dataset_per_section\layers_split\EGFP\info
Patched: D:\BCH-cjDAE8-mHGHpA\cjSwan\cjSwan-confocal-endogenous_neuroglancer_dataset_per_section\layers_split\dTom\info
Patched: D:\BCH-cjDAE8-mHGHpA\cjSwan\cjSwan-confocal-endogenous_neuroglancer_dataset_per_section\layers_split\aTH\info


# **Serve locally & open in Neuroglancer**


## Serve folder with desired Neuroglancer dataset using **cmd**:


### cjSwan Ab-Amplified VS200 dataset
```bat
cd /d D:\BCH-cjDAE8-mHGHpA\cjSwan\VS200-10X\cjSwan-Amplified_neuroglancer_precomputed\layers_split
http-server . -p 8080 --cors
```

### cjSwan Endogenous VS200 dataset
```bat
cd /d F:\cjSwan-Endogenous_neuroglancer_precomputed\layers_split
http-server . -p 8080 --cors
```

### cjSwan Ab-Amplified confocal dataset
```bat
cd /d D:\BCH-cjDAE8-mHGHpA\cjSwan\confocal-amplified_neuroglancer_dataset_per_section\layers_split
http-server . -p 8080 --cors
```

### cjSwan Endogenous confocal dataset

```bat
cd /d D:\BCH-cjDAE8-mHGHpA\cjSwan\cjSwan-confocal-endogenous_neuroglancer_dataset_per_section\layers_split
http-server . -p 8080 --cors
```

### cjDove Ab-Amplified confocal dataset
```bat
cd /d D:\cjDove-DAE8-Systemic\cjDove-confocal_neuroglancer_precomputed\layers_split
http-server . -p 8080 --cors
```

### Mouse Systemic 5E+10 vg dose
```
cd /d D:\BCH-cjDAE8-mHGHpA\Mouse_Systemic\Systemic-Injection_5E10\MouseSystemic_5E10_neuroglancer_precomputed\layers_split
http-server . -p 8080 --cors
```

### Mouse Systemic 5E+11 vg dose
```
cd /d D:\BCH-cjDAE8-mHGHpA\Mouse_Systemic\Systemic-Injection_5E11\MouseSystemic_5E11_neuroglancer_precomputed\layers_split
http-server . -p 8080 --cors
```

### Mouse Local (5E+11 vg/mL) 300 µL dose
```
cd /d D:\BCH-cjDAE8-mHGHpA\Mouse_Systemic\Systemic-Injection_5E11\MouseSystemic_5E11_neuroglancer_precomputed\layers_split
http-server . -p 8080 --cors
```

### Neuroglancer URL (DAPI, aGFP, aRFP, aTH)
```
https://neuroglancer-demo.appspot.com/#!{
  "layers": {
    "DAPI": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/DAPI/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(0.0, normalized(), normalized())); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    },
    "aGFP": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/aGFP/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(0.0, normalized(), 0.0)); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    },
    "aRFP": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/aRFP/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(normalized(), 0.0, 0.0)); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    },
    "aTH": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/aTH/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(0.0, 0.0, normalized())); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    }
  }
}

```

### Neuroglancer URL (DAPI, EGFP, dTom, aTH)
```
https://neuroglancer-demo.appspot.com/#!{
  "layers": {
    "DAPI": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/DAPI/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(0.0, normalized(), normalized())); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    },
    "EGFP": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/EGFP/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(0.0, normalized(), 0.0)); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    },
    "dTom": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/dTom/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(normalized(), 0.0, 0.0)); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    },
    "aTH": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/aTH/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(0.0, 0.0, normalized())); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    }
  }
}

```

### Neuroglancer URL (Nissl, aTH, dTom, aRFP)
```
https://neuroglancer-demo.appspot.com/#!{
  "layers": {
    "Nissl": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/Nissl/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(0.0, normalized(), normalized())); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    },
    "aTH": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/aTH/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(0.0, normalized(), 0.0)); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    },
    "dTom": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/dTom/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(normalized(), 0.0, 0.0)); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    },
    "aRFP": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/aRFP/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ float v = normalized(); emitRGB(vec3(v, 0.0, v)); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    }
  }
}

```

### Neuroglancer URL (DAPI, aTH, dTom, aRFP)
```
https://neuroglancer-demo.appspot.com/#!{
  "layers": {
    "DAPI": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/DAPI/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(0.0, normalized(), normalized())); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    },
    "aTH": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/aTH/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(0.0, normalized(), 0.0)); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    },
    "dTom": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/dTom/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ emitRGB(vec3(normalized(), 0.0, 0.0)); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    },
    "aRFP": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/aRFP/",
      "shader": "#uicontrol invlerp normalized\nvoid main(){ float v = normalized(); emitRGB(vec3(v, 0.0, v)); }",
      "shaderControls": { "normalized": { "range": [0, 65535] } },
      "blend": "additive",
      "opacity": 1.0
    }
  }
}

```